In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow_hub

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import pandas as pd

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Version:  2.3.1
Eager mode:  True
Hub version:  0.9.0
GPU is NOT AVAILABLE


In [34]:
dataset = pd.read_csv('train.csv')
print(dataset.head())
X = dataset.iloc[:,1].values

from sklearn.impute import SimpleImputer 
#print(humor_data.isnull().sum())
constant_imputer=SimpleImputer(strategy='constant', fill_value=0)

dataset.iloc[:]=constant_imputer.fit_transform(dataset)
Y = dataset.iloc[:,4].values

print(Y)

   id                                               text  is_humor  \
0   1  TENNESSEE: We're the best state. Nobody even c...         1   
1   2  A man inserted an advertisement in the classif...         1   
2   3  How many men does it take to open a can of bee...         1   
3   4  Told my mom I hit 1200 Twitter followers. She ...         1   
4   5  Roses are dead. Love is fake. Weddings are bas...         1   

   humor_rating  humor_controversy  offense_rating  
0          2.42                1.0             0.2  
1          2.50                1.0             1.1  
2          1.95                0.0             2.4  
3          2.11                1.0             0.0  
4          2.78                0.0             0.1  
[1. 1. 0. ... 0. 0. 0.]


In [35]:
train_examples, test_examples, train_labels, test_labels = train_test_split(X,Y, test_size=0.2,random_state=0)
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))
print("TRAIN Dataset -> ")
print(train_examples[:10])
print(" Test Dataset ->")
print(train_labels[:10])

Training entries: 6400, test entries: 1600
TRAIN Dataset -> 
['Better to have loved and lost, than to have never loved at all.'
 "I left my Adderall in my Ford Fiesta. Now it's a @FordFocus"
 "My wife and I just made a porno. Too bad she's my late wife."
 'Life consists not in holding good cards, but in playing those you hold well.'
 '"Your tongue is germ free only if it is pink. If it is white there is a thin film of bacteria on it"'
 "My wife was excited when I blindfolded her in the bedroom... until I spun her around 10 times and let her know we're playing pin the tail on the donkey."
 'Do deaf girls masturbate with one hand so they can moan with the other?'
 "People who support the Endangered Species Act haven't tasted my Bald Eagle Souffle"
 'What do you get when you mix a rhinoceros with an elephant? Elephino...'
 "Why don't feminists work cash registers? They can't make change."]
 Test Dataset ->
[0. 1. 1. 0. 0. 1. 1. 1. 1. 1.]


In [36]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [37]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_examples)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_examples)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_examples)
test_padded = pad_sequences(test_sequences,maxlen=max_length)

In [38]:
#Building the LSTM Model
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [39]:
model_lstm.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32)                4224      
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 198       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 164,429
Trainable params: 164,429
Non-trainable params: 0
_________________________________________________________________


In [40]:
model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
num_epochs = 30
history2 = model_lstm.fit(padded, train_labels, batch_size=1000,epochs=num_epochs, validation_data=(test_padded, test_labels))

Epoch 1/30
7/7 [==============================] - 3s 374ms/step - loss: 0.6911 - accuracy: 0.6550 - val_loss: 0.6878 - val_accuracy: 0.6994
Epoch 2/30
7/7 [==============================] - 2s 309ms/step - loss: 0.6859 - accuracy: 0.6900 - val_loss: 0.6814 - val_accuracy: 0.6994
Epoch 3/30
7/7 [==============================] - 2s 342ms/step - loss: 0.6789 - accuracy: 0.6900 - val_loss: 0.6725 - val_accuracy: 0.6994
Epoch 4/30
7/7 [==============================] - 2s 253ms/step - loss: 0.6686 - accuracy: 0.6900 - val_loss: 0.6577 - val_accuracy: 0.6994
Epoch 5/30
7/7 [==============================] - 2s 268ms/step - loss: 0.6511 - accuracy: 0.6900 - val_loss: 0.6328 - val_accuracy: 0.6994
Epoch 6/30
7/7 [==============================] - 2s 261ms/step - loss: 0.6273 - accuracy: 0.6900 - val_loss: 0.6130 - val_accuracy: 0.6994
Epoch 7/30
7/7 [==============================] - 2s 256ms/step - loss: 0.6170 - accuracy: 0.6900 - val_loss: 0.6107 - val_accuracy: 0.6994
Epoch 8/30
7/7 [====

In [41]:
codalab_data = pd.read_csv('public_dev.csv')
test_data_codalab = codalab_data['text'].values

In [42]:
tokenizer1 = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer1.fit_on_texts(test_data_codalab)
word_index = tokenizer1.word_index
sequences1 = tokenizer1.texts_to_sequences(test_data_codalab)
padded1 = pad_sequences(sequences1,maxlen=max_length, truncating=trunc_type)
print(padded1)
# humor = model_conv.predict_classes(padded1)
humor = model_lstm.predict_classes(padded1)
# for i in range(len(humor)):
#     print(humor[i][0])

[[   0    0    0 ...    4    2 1618]
 [   0    0    0 ...  453    2  993]
 [   0    0    0 ...   38 1623  109]
 ...
 [   0    0    0 ...    3  326 4162]
 [   0    0    0 ...   51 1390  897]
 [   0    0    0 ...  153   51 4168]]


In [43]:
for i in range(len(humor)):
    print(humor[i][0])

0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
0
1
0
1
1
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
1
0
1
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
1
0
0
0
0
1
0
1
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
1
0
1
0
1
0
0
0
0
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
1
0
0
0
1
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
1
0
1
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
1
1
0
0
1
1
0
1
0
0
1
0
0
1
